In [1]:
# Login part
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

PATH = "C:\\Users\\ccuuttww\\Desktop\\Chrome_Driver\\chromedriver.exe"
driver_ = webdriver.Chrome(PATH)
driver_.set_window_size(1024, 1000)

def driver_get(url):
    driver_.get(url)
    
    return driver_


def get_element(driver, method, name, obj):   
    element = eval(obj+".find"+method+"(\""+name+"\")")
    
    return element


def action_click(driver,element):
    actions = ActionChains(driver) 
    actions.click(element)
    actions.perform()    
    
    return


def login(url, username, password, name_list = {0: "btn-email", 1: "email", 2: "password"}):
    
    driver = driver_get(url)


    try:
       
        time.sleep(15)
        sign_in_tab = get_element(driver,"_element_by_class_name",name_list[0],"driver")
        action_click(driver,sign_in_tab)
        
        time.sleep(2)       
        login_username = get_element(driver,"_element_by_name",name_list[1],"driver")
        login_username.send_keys(username)
        
        login_password = get_element(driver,"_element_by_name",name_list[2],"driver")
        login_password.send_keys(password)
        login_password.send_keys(Keys.RETURN)  

    except:
        
        print("login error")
        driver.quit()
        
    
    return


def readcsv(file):
    category = pd.read_csv(file, encoding = "utf-8")
    
    return category


# load resources dict
def load_url_index(csv):    
    category = readcsv(csv)    
    for cat,index in zip(category["Resources"],category["index"]):        
        resources[cat] = index 
        
    return resources
    

# get the facility in the main page such as plantation
def get_facility(driver, classname):
           
    return get_element(driver, "_elements_by_class_name",classname["land"],"driver")


# main function
def operate(url, purchase, resources, plans, classname):
    try:
        
    
        driver = driver_get(url)
        time.sleep(5)
    
        for i in range(len(get_element(driver, "_elements_by_class_name",classname["land"],"driver"))):
        
            # select all dacility in main page
            lands = get_element(driver, "_elements_by_class_name",classname["land"],"driver")
        
            # get facility name 
            facility = lands[i].find_elements_by_class_name(classname["facility_name"])
            #print(facility[0].text)
        
            for plan in plans:
            
                # buy
                if facility[0].text == plan and plans[plan]["action"]=="buy":
                
                    actions = ActionChains(driver)
                
                    # go to selected facility
                    actions.click(lands[i])
                    actions.perform()
                    time.sleep(3)
                    buy(purchase,resources,"https://www.simcompanies.com/market/resource/")
                    time.sleep(3)
                    break
                
                # farm  
                if facility[0].text == plan and plans[plan]["action"]=="farm":
                
                    if facility[1].text == "LVL " + plans[plan]["level"]:
                    
                        actions = ActionChains(driver)
                    
                        # go to selected facility 
                        actions.click(lands[i])
                        actions.perform()
                    
                        # wait for load
                        time.sleep(5)
                        farm(driver,plan,classname)
                    
                        # wait for load
                        time.sleep(3)                
                        break
    except:
        
        print("operate error")
        driver.quit()
        
    # all finish    
    driver.quit()    
        

def buy(purchase,resources,url):
    
    try:
    
        for unit in purchase:
        
            # the items url with index from csv
            f_url = url + str(resources[unit])
            driver = driver_get(f_url)
            time.sleep(5)
            buy = get_element(driver, "_element_by_name","quantity","driver")
            buy.clear()
            buy.send_keys(purchase[unit]["amount"])
            buy.send_keys(Keys.RETURN)
    
        # beware "MAP" must be shown on pages or its will be fail
        Main_Page = get_element(driver, "_element_by_class_name","test-navigation-map","driver")
        actions = ActionChains(driver)
        actions.click(Main_Page)
        actions.perform()
        
    except:
        
        print("buy error")
        #driver.quit()
        
    return
  
        
def farm(driver,plan,classname):
    
    try:
    
        # all items block
        block = get_element(driver, "_elements_by_class_name",classname["farm_block"],"driver")
    
        for i in range(len(block)):
        
            texts = block[i].find_elements_by_class_name(classname["farm_corps_name"])
        
            if texts[0].text == plans[plan]["corps"]:
            
                farm = block[i].find_elements_by_name("amount")
                farm[0].clear()
                farm[0].send_keys(plans[plan]["amount"])
                break
            
        # for visual confirm      
        time.sleep(1)
        farm[0].send_keys(Keys.RETURN)  

    except:
        
        print("farm error")
        #driver.quit()
        
    return



# either buy or farm
plans = {"PLANTATION":{"level":"2","corps":"APPLES","amount":1,"action":"farm"},
         "FARM":{"level":"2","corps":"EGGS","amount":5,"action":"farm"},
         "EXCHANGE":{"action":"buy"},
        }


# amount of resources to purchase
purchase = {"Water":{"amount":10},
            "Seeds":{"amount":10}
           }


# resources with url index
resources = dict()


# key : resources, index : number
resources = load_url_index("category.csv")
classname = {"facility_name":"iiBSXG", "land":"iPLrNb", "farm_block":"IbSCg", "farm_corps_name":"eiemH"}


User_infor = readcsv("User_infor.csv")
login("https://www.simcompanies.com/signin/",User_infor["Username"],User_infor["Password"]) 
operate("https://www.simcompanies.com/landscape", purchase, resources, plans, classname)

In [ ]:
# support function
# capture items index from echange
import re
import csv
import pandas as pd

reso = {}

def add(self, key, value):
    self[key] = value

    
def find_resources_list(exchange):
    for item in exchange:
        category = item.find_elements_by_tag_name("img")
        index = re.findall('(\d+)',item.get_attribute("href"))
        add(reso,category[0].get_attribute("alt"),index[0])       
        #print("\""+category[0].get_attribute("alt")+"\":"+index[0]+",") 
        
    return

        
def write_csv(reso):
    csv_file = open(f'Category.csv', 'w', encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Resources','index'])    
    for i in reso:
        csv_writer.writerow([i,str(reso[i])])   
        
    return
    
def readcsv(file):
    category = pd.read_csv(file, encoding = "utf-8")
    return category      


driver_get("https://www.simcompanies.com/market/resource")
time.sleep(3)
exchange = driver_.find_elements_by_class_name("xmYII")


# find all ID
find_resources_list(exchange)


# write it into CSV
write_csv(reso)


# read CSV
category = readcsv("Category.csv")
category.head()

In [ ]:
#driver.current_url